## Let's start by setting up the environment.

In [ ]:
import sys
from os import path
sys.path.append("../")
import os
import im

os.environ['AWS_DEFAULT_REGION']='us-west-2'

ims = im.Session()

# Replace with a role that gives IM access to s3 and cloudwatch
role='IMRole'

## Downloading test and training data

In [ ]:
import utils
from tensorflow.contrib.learn.python.learn.datasets import mnist
import tensorflow as tf

data_sets = mnist.read_data_sets('data', dtype=tf.uint8, reshape=False, validation_size=5000)

utils.convert_to(data_sets.train, 'train', 'data')
utils.convert_to(data_sets.validation, 'validation', 'data')
utils.convert_to(data_sets.test, 'test', 'data')

## Uploading the data

In [ ]:
inputs = ims.upload_data(path='data', key_prefix='data/mnist')

# Complete source code

In [ ]:
!cat 'mnist.py'

# Running TensorFlow training on IM

We can use the SDK to run our local training script on IM infrastructure.

1. Pass the path to the abalone.py file, which contains the functions for defining your estimator, to the im.TensorFlow init method.
2. Pass the S3 location that we uploaded our data to previously to the fit() method.

In [ ]:
from im.tensorflow import TensorFlow

mnist_estimator = TensorFlow(entry_point='mnist.py',
                                role=role,
                                hyperparameters={'training_steps' : 1000, 'evaluation_steps' : 100},
                                train_instance_count=2,
                                train_instance_type='ml.p2.xlarge')

mnist_estimator.fit(inputs)

# Deploy the trained model to prepare for predictions

The deploy() method creates an endpoint which serves prediction requests in real-time.

In [ ]:
mnist_predictor = mnist_estimator.deploy(initial_instance_count=1,
                                             instance_type='ml.c4.xlarge')

# Invoking the endpoint

In [ ]:
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

for i in range(10):
    data = mnist.test.images[i].tolist()
    tensor_proto = tf.make_tensor_proto(values=np.asarray(data), shape=[1, len(data)], dtype=tf.float32)
    predict_response = mnist_predictor.predict(tensor_proto)
    
    print("========================================")
    label = np.argmax(mnist.test.labels[i])
    print("label is {}".format(label))
    prediction = predict_response['outputs']['classes']['int64Val'][0]
    print("prediction is {}".format(prediction))

# Deleting the endpoint

In [ ]:
mnist_estimator.delete_endpoint()